In [12]:
def preprocessing(df):
    df.bidPrice=df.loc[:,'bidPrice'].replace(to_replace=0, method='ffill')
    df.bidSize=df.loc[:,'bidSize'].replace(to_replace=0, method='ffill')
    df.askPrice=df.loc[:,'askPrice'].replace(to_replace=0, method='ffill')
    df.askSize=df.loc[:,'askSize'].replace(to_replace=0, method='ffill')
    df=df.dropna()
    # to exclude 0
    df=df[df['bidPrice']>df.bidPrice.mean()-df.bidPrice.std()]
    df=df[df['askPrice']>df.askPrice.mean()-df.askPrice.std()]
    df['mid']=(df.askPrice+df.bidPrice)/2
    df['vwap']=((df.loc[:,'bidPrice']*df.loc[:,'bidSize'])+(df.loc[:,'askPrice']*df.loc[:,'askSize']))/(df.loc[:,'bidSize']+df.loc[:,'askSize'])
    df['spread']=df.vwap-(df.askPrice+df.bidPrice)/2
    df['v']=(df.askPrice+df.bidPrice)/2-((df.askPrice+df.bidPrice)/2).shift(60)
    df['return']=(df.askPrice/df.bidPrice.shift(1))-1
    df['sigma']=df.spread.rolling(60).std()
    return df

In [2]:
import zmq
import datetime
import pandas as pd
import numpy as np
import numpy
from numpy import inf

import json
import tpqib
import datetime

import matplotlib.pyplot as plt

iterations = 0
df = pd.DataFrame()

context = zmq.Context()

#Forwarding ML output
socket_pub = context.socket(zmq.PUB)
socket_pub.bind('tcp://127.0.0.1:7010')

port = "7000"
# socket to talk to server
socket_sub = context.socket(zmq.SUB)
print ("Collecting & plotting stock prices.")
socket_sub.connect("tcp://localhost:%s" % port)

socket_sub.setsockopt_string(zmq.SUBSCRIBE, u'SPY')

In [14]:
## storing ML output
df_ml=pd.DataFrame()
#window=100
#for _ in range(window):
while True:
    iterations += 1
    string = socket_sub.recv_string()
    sym, bidPrice,bidSize,askPrice,askSize = string.split()
    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))
    dt = datetime.datetime.now()
    df = df.append(pd.DataFrame({'Stock':sym,'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))
    data=preprocessing(df)
    x = data.to_string(header=False,index=False,index_names=False).split('\n')
    socket_pub.send_string(x[-1])
    print(x[-1])

SPY  249.09  169.0  249.08   98.0  249.085  249.086330  1.329588e-03  4.000000e-02  0.000080  0.005368
SPY  249.09  169.0  249.08    2.0  249.085  249.089883  4.883041e-03  3.500000e-02  0.000040  0.005388
SPY  249.09  169.0  249.08    2.0  249.085  249.089883  4.883041e-03  3.500000e-02  0.000040  0.005406
SPY  249.09  185.0  249.08    2.0  249.085  249.089893  4.893048e-03  2.500000e-02  0.000040  0.005420
SPY  249.09  185.0  249.08   10.0  249.085  249.089487  4.487179e-03  2.500000e-02  0.000040  0.005417
SPY  249.09   21.0  249.08   10.0  249.085  249.086774  1.774194e-03  2.500000e-02  0.000040  0.005398
SPY  249.09   21.0  249.06   10.0  249.075  249.080323  5.322581e-03  1.500000e-02  0.000040  0.005405
SPY  249.09   21.0  249.06   27.0  249.075  249.073125 -1.875000e-03  1.000000e-02  0.000120  0.005417
SPY  249.07   21.0  249.06   27.0  249.065  249.064375 -6.250000e-04  0.000000e+00  0.000040  0.005399
SPY  249.07    2.0  249.06   27.0  249.065  249.060690 -4.310345e-03  0.0

KeyboardInterrupt: 